<a href="https://colab.research.google.com/github/Vivek-Hiremath55/AI_Systems_Pro/blob/main/VivekHiremath/Inference_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Requirements

### Data Classifier Probabilities + Image Classifier Probabilities

#### nstructions
1. Load Data Classifier Model
2. Load image Classifier Model


cumulative_column_list = ['isic_id', 'age_approx', 'sex', 'clin_size_long_diam_mm', 'tbp_lv_area_perim_ratio',
               'tbp_lv_color_std_mean', 'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_x',
               'tbp_lv_y', 'tbp_lv_z', 'target']

#### Requirements for Data Model ->
1. All except 'isic_id'
2. Drop null values
3. One Hot encoding for categorical variables
4. Make sure the columns in the dataframe are the same as those the model is trained on

#### Requirement for Image Model ->
1. Columns needed 'isic_id' 'target'



# Importing Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib
import pandas as pd
import numpy as np
import torch.nn as nn
from torchvision.models import resnet18
import torch.optim as optim
from IPython.display import display
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch
from torch.utils.data import TensorDataset, DataLoader

# Importing Classification Models

In [3]:
data_model = joblib.load('/content/drive/MyDrive/AIS_Pro_Data/best_rf_model_ADASYN.pkl')
image_model = joblib.load('/content/drive/MyDrive/AIS_Pro_Data/image_classifier.pkl')

# Importing Dataset

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/AIS_Pro_Data/train-metadata.csv')

<ipython-input-4-9d394cdcbe96>:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/content/drive/MyDrive/AIS_Pro_Data/train-metadata.csv')


# Tranforming Main Dataset

In [5]:
cumulative_column_list = ['isic_id', 'age_approx', 'sex', 'clin_size_long_diam_mm', 'tbp_lv_area_perim_ratio',
                          'tbp_lv_color_std_mean', 'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_x',
                          'tbp_lv_y', 'tbp_lv_z', 'target']

In [6]:
cumulative_data = train_data[cumulative_column_list]

In [7]:
cumulative_data.dropna(axis = 0, inplace =True)

<ipython-input-7-5a93b13e6883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_data.dropna(axis = 0, inplace =True)


In [8]:
cumulative_data.columns


Index(['isic_id', 'age_approx', 'sex', 'clin_size_long_diam_mm',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'target'],
      dtype='object')

In [9]:
cumulative_data.head(1)

,isic_id,age_approx,sex,clin_size_long_diam_mm,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_x,tbp_lv_y,tbp_lv_z,target
0,ISIC_0015670,60.0,male,3.04,27.47617,0.0,0.901302,Right Leg - Upper,-182.703552,613.493652,-42.427948,0


In [10]:
columns_for_encoding = ['sex', 'tbp_lv_location']

In [11]:
cumulative_data_comp = pd.get_dummies(cumulative_data[columns_for_encoding], drop_first =True)

In [12]:
cumulative_data_comp.head(1)

,sex_male,tbp_lv_location_Left Arm,tbp_lv_location_Left Arm - Lower,tbp_lv_location_Left Arm - Upper,tbp_lv_location_Left Leg,tbp_lv_location_Left Leg - Lower,tbp_lv_location_Left Leg - Upper,tbp_lv_location_Right Arm,tbp_lv_location_Right Arm - Lower,tbp_lv_location_Right Arm - Upper,...,tbp_lv_location_Right Leg - Lower,tbp_lv_location_Right Leg - Upper,tbp_lv_location_Torso Back,tbp_lv_location_Torso Back Bottom Third,tbp_lv_location_Torso Back Middle Third,tbp_lv_location_Torso Back Top Third,tbp_lv_location_Torso Front,tbp_lv_location_Torso Front Bottom Half,tbp_lv_location_Torso Front Top Half,tbp_lv_location_Unknown
0,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [13]:
combined_df = pd.concat([cumulative_data, cumulative_data_comp], axis = 1)

In [14]:
combined_df.head(1)

,isic_id,age_approx,sex,clin_size_long_diam_mm,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_x,tbp_lv_y,...,tbp_lv_location_Right Leg - Lower,tbp_lv_location_Right Leg - Upper,tbp_lv_location_Torso Back,tbp_lv_location_Torso Back Bottom Third,tbp_lv_location_Torso Back Middle Third,tbp_lv_location_Torso Back Top Third,tbp_lv_location_Torso Front,tbp_lv_location_Torso Front Bottom Half,tbp_lv_location_Torso Front Top Half,tbp_lv_location_Unknown
0,ISIC_0015670,60.0,male,3.04,27.47617,0.0,0.901302,Right Leg - Upper,-182.703552,613.493652,...,False,True,False,False,False,False,False,False,False,False


In [15]:
combined_df.drop(columns = ['sex', 'tbp_lv_location'], axis = 1 , inplace = True)

In [16]:
combined_df.head(1)

,isic_id,age_approx,clin_size_long_diam_mm,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_eccentricity,tbp_lv_x,tbp_lv_y,tbp_lv_z,target,...,tbp_lv_location_Right Leg - Lower,tbp_lv_location_Right Leg - Upper,tbp_lv_location_Torso Back,tbp_lv_location_Torso Back Bottom Third,tbp_lv_location_Torso Back Middle Third,tbp_lv_location_Torso Back Top Third,tbp_lv_location_Torso Front,tbp_lv_location_Torso Front Bottom Half,tbp_lv_location_Torso Front Top Half,tbp_lv_location_Unknown
0,ISIC_0015670,60.0,3.04,27.47617,0.0,0.901302,-182.703552,613.493652,-42.427948,0,...,False,True,False,False,False,False,False,False,False,False


In [17]:
td_pos = combined_df[cumulative_data['target'] == 1]
td_neg = combined_df[cumulative_data['target'] == 0]

In [18]:
td_pos.shape, td_neg.shape

((381, 31), (387274, 31))

In [19]:
shuffle_td_neg = td_neg.sample(random_state = 42, n = 381)

In [20]:
td_complete = pd.concat([td_pos, shuffle_td_neg])
td_complete.shape

(762, 31)

# Transforming dataset for data model

In [21]:
data_model_data = td_complete.drop('isic_id', axis = 1) #Selecting required columns

In [22]:
data_model_data_comp = pd.get_dummies(data_model_data, drop_first = True)

In [23]:
if hasattr(data_model, 'feature_names_in_'):
    feature_names = data_model.feature_names_in_

In [24]:
for entry in data_model_data_comp:
    if entry not in feature_names and entry != 'target':
        data_model_data_comp.drop(entry, axis = 1, inplace = True)

In [25]:
len(feature_names), len(data_model_data_comp.columns)

(28, 29)

#### The extra is target

# Transforming dataset for Image Model

In [26]:
image_data = td_complete[['isic_id', 'target']]

In [27]:
import h5py
from PIL import Image
import numpy as np, pandas as pd
import os
import io
from IPython.display import display
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [28]:


# Define the path to your .hdf5 dataset file
hdf5_file_path = "/content/drive/MyDrive/AIS_Pro_Data/train-image.hdf5"

# Define the target image size (forexample 224x224 for ResNet)
target_size = (224, 224)

In [29]:
hdf5_file = h5py.File(hdf5_file_path, 'r')

In [30]:
images = []
# Reading image data and resizing the image so it fits resnet's required size

def image_processor(image_name):
    image_data = hdf5_file[image_name][()]
    image = Image.open(io.BytesIO(image_data))
    image_2 = image.resize(target_size)
    return image_2

In [31]:
# Accessing images using the image names from td_complete
images.clear()
for i in range(image_data.shape[0]):
    images.append(image_processor(image_data['isic_id'].iloc[i]))

# Prob Pred for Data Model

In [32]:
X = data_model_data_comp.drop('target', axis =1)
y = data_model_data_comp['target']

In [33]:
y_data_prob = data_model.predict_proba(X)

# Prob Pred for Image Model

In [64]:
Xi = images
yi = image_data['target']

In [65]:
from torchvision import transforms
import torch

# Define transformations for your images to match ResNet input requirements
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet requires 224x224 input size
    transforms.ToTensor(),  # Convert to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Apply transformation to each image in your list
X_transformed = [transform(image) for image in Xi]

# Stack the list into a single tensor
X_tensor = torch.stack(X_transformed)


In [66]:
from torch.utils.data import TensorDataset, DataLoader

# Convert target labels into a tensor
#y_tensor = torch.tensor(y_test.values)
y_tensor = torch.tensor(yi.values)
# Create a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [68]:
import numpy as np
import torch

image_model.eval()
y_image_prob = []  # List to store probabilities
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Get model outputs
        outputs = image_model(inputs)

        # Apply softmax to get probabilities
        probabilities = torch.softmax(outputs, dim=1)  # shape [batch_size, 2]
        probabilities = probabilities[:, 1]  # Get probabilities for the positive class

        # Collect probabilities
        y_image_prob.extend(probabilities.cpu().numpy())  # Add to the list

        # Convert probabilities to binary predictions
        predicted = (probabilities >= 0.5).float()

        # Flatten predicted and labels for comparison
        predicted_flat = predicted.view(-1)  # Flatten predicted
        labels_flat = labels.view(-1)  # Flatten labels to match

        total += labels_flat.size(0)
        correct += (predicted_flat == labels_flat).sum().item()  # Compare with actual labels

# Print accuracy
#print(f"Accuracy: {100 * correct / total}%")

# Now y_image_prob contains all output probabilities
#print("Collected probabilities:", y_image_prob)


In [69]:
len(y_data_prob), len(y_image_prob)

(762, 762)

# Training Model based on probabilities

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
lr_model = LogisticRegression()

In [42]:
y_data_prob.shape

(762, 2)

In [43]:
type(y_data_prob)

numpy.ndarray

In [48]:
y_data_prob[0][1]

0.9260746706753619

In [49]:
y_data_prob_list =[]

In [51]:
y_data_prob_list.clear()
for i in range(len(y_data_prob)):
  y_data_prob_list.append(y_data_prob[i][1])

In [61]:
y_image_prob[0:5]

[0.9097335, 0.26028273, 0.89759135, 0.00020162167, 0.008589776]

In [70]:

print(f"y_data_prob shape: {len(y_data_prob_list)}")
#print(f"y_image_prob shape: {y_image_prob.shape}")
print(f"y_image_prob shape: {len(y_image_prob)}")
print(f"target shape: {image_data['target'].values.shape}")

#y_data_prob_trimmed = y_data_prob[:762]  # Take the first 762 elements
y_data_prob_trimmed = y_data_prob_list  # Take the first 762 elements

target = image_data['target']

# Create a DataFrame
df = pd.DataFrame({
    'y_data_prob': y_data_prob_trimmed,
    'y_image_prob': y_image_prob,
    'target': target  # Ensure target is in the right format
})


y_data_prob shape: 762
y_image_prob shape: 762
target shape: (762,)


In [71]:
df.head()

,y_data_prob,y_image_prob,target
387,0.926075,0.860457,1
935,0.619048,0.016416,1
1245,0.836230,0.966905,1
1846,0.828447,0.810337,1
3478,0.884509,0.238706,1


In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X = df.drop('target', axis = 1)
y = df['target']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify=y)

In [75]:
lr_model.fit(X_train, y_train)

LogisticRegression()

In [76]:
y_pred_final = lr_model.predict(X_test)

In [77]:
from sklearn.metrics import confusion_matrix

In [78]:
confusion_matrix(y_test, y_pred_final)

array([[76,  1],
       [ 2, 74]])

In [79]:
y_pred_final_prob = lr_model.predict_proba(X_test)

In [80]:
from sklearn.metrics import confusion_matrix

# Assuming y_test contains true labels and y_pred_final contains predicted labels
cm = confusion_matrix(y_test, y_pred_final)

# Extract TP, TN, FP, FN
tn, fp, fn, tp = cm.ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # True Negative Rate

# Display results
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


Sensitivity: 0.97
Specificity: 0.99


In [81]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming df is already created as per your provided code
print(df.head())  # Check the first few rows

# Prepare features and target
X = df[['y_data_prob', 'y_image_prob']]  # Features
y = df['target']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model
dt_classifier.fit(X_train, y_train)

# Make predictions
y_pred = dt_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(cm)


      y_data_prob  y_image_prob  target
387      0.926075      0.860457       1
935      0.619048      0.016416       1
1245     0.836230      0.966905       1
1846     0.828447      0.810337       1
3478     0.884509      0.238706       1
Accuracy: 0.99
Confusion Matrix:
[[77  2]
 [ 0 74]]


In [85]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming df is already created as per your provided code
print(df.head())  # Check the first few rows

# Prepare features and target
X = df[['y_data_prob', 'y_image_prob']]  # Features
y = df['target']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model
dt_classifier.fit(X_train, y_train)

# Make predictions
y_pred = dt_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Extract values from confusion matrix
TN, FP, FN, TP = cm.ravel()  # Ensure you have a binary classification

# Calculate sensitivity and specificity
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(cm)
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")


      y_data_prob  y_image_prob  target
387      0.926075      0.860457       1
935      0.619048      0.016416       1
1245     0.836230      0.966905       1
1846     0.828447      0.810337       1
3478     0.884509      0.238706       1
Accuracy: 0.99
Confusion Matrix:
[[77  2]
 [ 0 74]]
Sensitivity: 1.00
Specificity: 0.97
